In [1]:
from datasets import load_dataset

ds = load_dataset("K-Monitor/kmdb_base")

/home/boa/sajtoadatbazis-automat/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds

DatasetDict({
    train: Dataset({
        features: ['news_id', 'title', 'description', 'text', 'source_url', 'archive_url', 'kmdb_url', 'newspaper', 'category', 'pub_time', 'persons', 'institutions', 'places', 'others', 'files'],
        num_rows: 59180
    })
})

In [5]:
from collections import defaultdict
from datetime import datetime
import pandas as pd

def process_articles(article_list):
    # Negyedéves adatok tárolása
    quarterly_data = defaultdict(lambda: defaultdict(int))

    for article in article_list:
        # Publikáció időpontjának ellenőrzése és beolvasása
        pub_time = article.get('pub_time', '')
        if not pub_time:
            continue
        
        try:
            pub_datetime = datetime.strptime(pub_time, '%Y-%m-%d %H:%M:%S')
        except ValueError:
            print(f"Warning: Date format mismatch for entry with news_id {article['news_id']}")
            continue

        # Év és negyedév számítása
        year = pub_datetime.year

        # Az aktához tartozó fájl lekérdezése
        for tag in article['others']:
            archive_name = tag if tag else 'Unknown'
            quarterly_data[archive_name][year] += 1

    # Az adatok átalakítása Pandas DataFrame-be
    df = pd.DataFrame(quarterly_data).T.fillna(0).astype(int)

    # Oszlopok elnevezése és rendezése időrend szerint
    df.columns = [f'{year}' for year in df.columns]
    df = df.reindex(sorted(df.columns, key=lambda x: int(x)), axis=1)

    return df


df = process_articles(ds['train'])

print(df)

IndexError: list index out of range

In [4]:
df.to_csv('tags_bontas.csv')